In [1]:
import dgl
import torch

Using backend: pytorch[15:02:52] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/phunc20/.virtualenvs/dglcpu/lib/python3.7/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.9.1.so: cannot open shared object file: No such file or directory



In [2]:
dataset = dgl.data.CoraGraphDataset()
dataset.num_classes

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


7

In [3]:
!ls ~/.dgl/cora_v2

cora_v2_dgl_graph.bin  ind.cora_v2.ally        ind.cora_v2.tx  ind.cora_v2.y
cora_v2_dgl_graph.pkl  ind.cora_v2.graph       ind.cora_v2.ty
ind.cora_v2.allx       ind.cora_v2.test.index  ind.cora_v2.x


How many graphs contained in `dataset`?

In [4]:
len(dataset)

1

In [5]:
# put the graph on GPU if there is one
if torch.cuda.is_available():
    g = dataset[0].to("cuda")
else:
    g = dataset[0]
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [6]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False]), 'train_mask': tensor([False, False, False,  ..., False, False, False])}
Edge features
{}


**(?)** What are those `Num.*Samples` sum up to? Their sum equals `1640`.
```
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
```

In [7]:
g.ndata["feat"].shape

torch.Size([2708, 1433])

In [8]:
[g.ndata[s].sum() for s in ["train_mask", "val_mask", "test_mask"]]

[tensor(140), tensor(500), tensor(1000)]

In [9]:
[g.ndata[s].shape for s in ["train_mask", "val_mask", "test_mask"]]

[torch.Size([2708]), torch.Size([2708]), torch.Size([2708])]

In [10]:
g.ndata["label"].shape

torch.Size([2708])

In [11]:
g.ndata["label"].unique()

tensor([0, 1, 2, 3, 4, 5, 6])

**(R)** So the `Num.*Samples` represents the number of nodes in train/val/test sets, but it seems that we do not take all nodes; we only take `1640/2708`.

In [12]:
from dgl.nn import GraphConv
import torch.nn as nn
import torch.nn.functional as F


class GCN(nn.Module):
    def __init__(self, num_in_feats, num_h_feats, num_classes):
        super(GCN, self).__init__() # (?) What's diff from super(self).__init__() or super().__init__()?
        self.conv1 = GraphConv(num_in_feats, num_h_feats)
        self.conv2 = GraphConv(num_h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [13]:
# put the model on GPU if there is one
if torch.cuda.is_available():
    model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes).to("cuda")
else:
    model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

It seems that PyTorch does not need a method like `.summary()` in TF to show the model's structure; instead, simply printing the model will show its structure.

In [14]:
model.extra_repr

<bound method Module.extra_repr of GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)>

In [15]:
model

GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

In [16]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for epoch in range(100):
        logits = model(g, features)
        pred = logits.argmax(1)      # a batch of predictions, so take argmax along axis=1
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 5 == 0:
            print(f"(epoch {epoch:03d})\n\tloss = {loss:.4f}\n\tval_acc = {val_acc:.4f} (best = {best_val_acc:.4f})\n\ttest_acc = {test_acc:.4f} (best = {best_test_acc:.4f})")

The next cell

- cannot run under my Arch Linux (cf. maybe <https://github.com/hfp/libxsmm>)
  ```
  DGLError: [18:11:38] /opt/dgl/src/array/cpu/./spmm_blocking_libxsmm.h:267: Failed to generate libxsmm kernel for the SpMM operation!
  ```
  - Mabye `libxsmm` needs to be manually installed and configured on Arch Linux
- runs fine on my Fedora 33

In [17]:
train(g, model)

(epoch 000)
	loss = 1.9451
	val_acc = 0.1760 (best = 0.1760)
	test_acc = 0.1900 (best = 0.1900)
(epoch 005)
	loss = 1.8834
	val_acc = 0.6000 (best = 0.6040)
	test_acc = 0.5940 (best = 0.6060)
(epoch 010)
	loss = 1.7932
	val_acc = 0.4440 (best = 0.6040)
	test_acc = 0.4530 (best = 0.6060)
(epoch 015)
	loss = 1.6799
	val_acc = 0.5360 (best = 0.6040)
	test_acc = 0.5540 (best = 0.6060)
(epoch 020)
	loss = 1.5446
	val_acc = 0.6440 (best = 0.6440)
	test_acc = 0.6470 (best = 0.6470)
(epoch 025)
	loss = 1.3906
	val_acc = 0.6860 (best = 0.6860)
	test_acc = 0.6830 (best = 0.6830)
(epoch 030)
	loss = 1.2237
	val_acc = 0.7180 (best = 0.7180)
	test_acc = 0.7150 (best = 0.7090)
(epoch 035)
	loss = 1.0518
	val_acc = 0.7380 (best = 0.7380)
	test_acc = 0.7310 (best = 0.7310)
(epoch 040)
	loss = 0.8840
	val_acc = 0.7520 (best = 0.7520)
	test_acc = 0.7470 (best = 0.7440)
(epoch 045)
	loss = 0.7287
	val_acc = 0.7700 (best = 0.7700)
	test_acc = 0.7500 (best = 0.7500)
(epoch 050)
	loss = 0.5919
	val_acc = 0.